#Introduction rapide aux Réseaux de Neurones avec Scikit-Learn

Dans cet exemple nous allons créer un Classifieur "Multi-Layer Perceptron" (plusieurs couches de neurones) afin de reconnaître des chiffres écrits à la main.

Tout d'abord, on fait l'import des bibliothèques et le téléchargement du dataset MNIST.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pnd

In [ ]:
!pip install -q fidle
import fidle


In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True,parser='auto')


MNIST est composé de miliers d'images avec des chiffres 0 à 9 écrits à la main. Ces images sont stockés sous la forme de tableaux de 28x28 pixels où chaque pixel a une valeur entre 0 et 255 (256 tons de gris).

Il est souvent conseillé de "normaliser" les données. Une manière simple dans notre cas est de diviser les valeurs par 255 afin d'avoir des valeurs réelles entre 0 et 1.

In [ ]:
X = X / 255.

Ici, nous allons séparer le dataset en deux groupes (train et test). Les données de chaque tableau sont représentés en tant qu'un simple array de 784 positions (28x28=784).

On profite également pour reinitialiser les index des tableaux.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,shuffle=False)
X_train=X_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

print (X.shape)
print(X_train.shape)
print(X_test.shape)

On peut visualiser chacun des chiffres, ainsi que son label, grâce au paragraphe suivant (par contre, on est obligés de reformater la sortie en 28x28 pixels). Ici, on affiche el 220ème chiffre dans le dataset d'entraînement.

In [ ]:
fidle.scrawler.images(X_train.values.reshape(-1,28,28), y_train,[220], x_size=4,y_size=4)

C'est le moment de créer notre réseau de neurones.

Nous allons créer un réseau d'une seule couche (`hidden_layer_sizes=(20,)`).
Egalement, il est indiqué que l'entrainement se fera sur 10 itérations, où chaque itération (*epoch*) refait un passage sur les données afin d'améliorer l'entraînement.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(20,),
    max_iter=10,
    alpha=1e-4,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate_init=0.2,
)

In [ ]:
mlp.fit(X_train, y_train)

Après l'étape d'entraînement (`fit()`), nous pouvons voir la précision du modèle sur le dataset d'entraînement et sur celui de test.

On obtient plus de 95% de précision, ce qui n'est pas mal pour un début.

In [ ]:
print(f"Training set score: {mlp.score(X_train, y_train):.3f}")
print(f"Test set score: {mlp.score(X_test, y_test):.3f}")

Chacun des 20 neurones aura une vue lègerement différente des autres. Certains ne seront même pas capables de reconnaître quoi que ce soit.

Le paragraphe suivant montre les poids (filtres) de chacun de ces neurones, après entraînement.

In [ ]:
fig, axes = plt.subplots(4, 5)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

Après l'entraînement, nous pouvons utiliser le modèle pour faire la prédiction sur le dataset `test`.

In [ ]:
y_pred = mlp.predict(X_test)


Le paragraphe suivant affiche 50 chiffres avec la valeur prédite en bas. Sauf rares exceptions, tous sont bien classés.

In [ ]:
y_pred=pnd.Series(y_pred)
fidle.scrawler.images(X_test.values.reshape(-1,28,28), y_pred,range(50,100), columns=12, x_size=1, y_size=1)

Ici, on s'intéresse juste aux erreurs de classement. Comme on avait accès aux labels `y_test`, on peut les comparer avec les prédictions de `y_pred`. Pour chaque image, il est indiqué la valeur prédite et, entre parenthèses, la valeur attendue.

In [ ]:
errors=[ i for i in range(len(X_test)) if y_pred[i]!=y_test[i] ]
errors=errors[:min(24,len(errors))]
fidle.scrawler.images(X_test.values.reshape(-1,28,28), y_test, errors, columns=6, x_size=1, y_size=1, y_pred=y_pred)

Cet exemple permet de faire ses premiers pas avec un réseau de neurones. Toutefois, Scikit Learn n'est pas la bibliothèque la plus adaptée car elle n'est pas adaptée à l'usage de dispositifs performants (GPU, par exemple). Pour aller plus loin, vous pouvez vous informer sur la bibliothèque [Keras](https://keras.io/keras_3/), qui vous permet d'écrire un code assez similaire à Scikit Learn tout en faisant appel à la puissance de Tensorflow ou Pytorch (deux bibliothèques "bas niveau" spécialement conçues pour l'apprentissage profond).